# Initialization

Import Dependencies

In [13]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import nltk
import math
import operator
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
import string
import json
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
np.random.seed(2018)
nltk.download('wordnet')
import random

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Preprocessing data

Stemming and tokenization

In [14]:
'''
Preprocessing function - Porter stemmer
'''
def preprocess_docs(data):
    processed_data=[]
    stemmer = PorterStemmer()
    for d in data:
        tempf=[]
        temp = d
        temp = ''.join(c for c in temp if c not in string.punctuation)
        temp = nltk.word_tokenize(temp)
        temp = [w.lower() for w in temp]
        temp = [t for t in temp if t not in nltk.corpus.stopwords.words('english')]
        for word in temp:
          tempf.append(stemmer.stem(word))
        processed_data.append(tempf)
    return processed_data

Save my files and load

In [15]:
import pickle

#save objects
def store_data():

  pickle_out = open("/content/drive/My Drive/IR Assignment/saved objects/comb_news.pickle","wb")
  pickle.dump(comb_news, pickle_out)
  pickle_out.close()

  pickle_out = open("/content/drive/My Drive/IR Assignment/saved objects/review.pickle","wb")
  pickle.dump(review, pickle_out)
  pickle_out.close()

  pickle_out = open("/content/drive/My Drive/IR Assignment/saved objects/master_dict.pickle","wb")
  pickle.dump(master_dict, pickle_out)
  pickle_out.close()

  pickle_out = open("/content/drive/My Drive/IR Assignment/saved objects/total_reviews.pickle","wb")
  pickle.dump(total_reviews, pickle_out)
  pickle_out.close()

  pickle_out = open("/content/drive/My Drive/IR Assignment/saved objects/dictionary.pickle","wb")
  pickle.dump(dictionary, pickle_out)
  pickle_out.close()

  pickle_out = open("/content/drive/My Drive/IR Assignment/saved objects/lda_model.pickle","wb")
  pickle.dump(lda_model, pickle_out)
  pickle_out.close()

  pickle_out = open("/content/drive/My Drive/IR Assignment/saved objects/doc_lda.pickle","wb")
  pickle.dump(doc_lda, pickle_out)
  pickle_out.close()

  pickle_out = open("/content/drive/My Drive/IR Assignment/saved objects/links.pickle","wb")
  pickle.dump(links, pickle_out)
  pickle_out.close()

#store_data()

#initialising objects
# temp_comb_news = []
# temp_review = []
# temp_master_dict = {}
# temp_total_reviews = 1
# temp_dictionary = {}
# temp_lda_model
# temp_doc_lda = {}

#load objects
def load_data():

  pickle_in = open("/content/drive/My Drive/IR Assignment/saved objects/comb_news.pickle","rb")
  temp_comb_news = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("/content/drive/My Drive/IR Assignment/saved objects/review.pickle","rb")
  temp_review = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("/content/drive/My Drive/IR Assignment/saved objects/master_dict.pickle","rb")
  temp_master_dict = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("/content/drive/My Drive/IR Assignment/saved objects/total_reviews.pickle","rb")
  temp_total_reviews = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("/content/drive/My Drive/IR Assignment/saved objects/dictionary.pickle","rb")
  temp_dictionary = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("/content/drive/My Drive/IR Assignment/saved objects/lda_model.pickle","rb")
  temp_lda_model = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("/content/drive/My Drive/IR Assignment/saved objects/doc_lda.pickle","rb")
  temp_doc_lda = pickle.load(pickle_in)
  pickle_in.close()

  pickle_in = open("/content/drive/My Drive/IR Assignment/saved objects/links.pickle","rb")
  temp_links = pickle.load(pickle_in)
  pickle_in.close()

  return temp_comb_news,temp_review,temp_master_dict,temp_total_reviews,temp_dictionary,temp_lda_model,temp_doc_lda,temp_links

comb_news,review,master_dict,total_reviews,dictionary,lda_model,doc_lda,links = load_data()

# comb_news = temp_comb_news
# review = temp_review
# master_dict = temp_master_dict
# total_reviews = temp_total_reviews
# dictionary = temp_dictionary
# lda_model = temp_lda_model
# doc_lda = temp_doc_lda


In [7]:
#dummy cell
print(1)

1


# Extracting results from query

Function to get results

In [16]:
def getResults(myquery):

  sample_query = [myquery]

  #preprocessing query before evaulation
  sample_query = preprocess_docs(sample_query)
  query = sample_query[0]
  print(query)

  query_dict = {}

  for query_term in query:

  # query term should be present in my master dictionary
    if query_term in master_dict:

      if query_term in query_dict:
        query_dict[query_term]['tf'] += 1
        query_dict[query_term]['df'] = len(master_dict[query_term])
        query_dict[query_term]['tf-idf'] = (1+math.log10(query_dict[query_term]['tf']))*(math.log10(total_reviews/query_dict[query_term]['df']))
      else:
        query_dict.update({query_term:{'tf':1, 'df':0, 'tf-idf':0.1}})
        query_dict[query_term]['df'] = len(master_dict[query_term])
        query_dict[query_term]['tf-idf'] = (1+math.log10(query_dict[query_term]['tf']))*(math.log10(total_reviews/query_dict[query_term]['df']))


  #normalise tf-idf for query
  q_norm_factor = 0;

  for term in query_dict:
    q_norm_factor += query_dict[term]['tf-idf']*query_dict[term]['tf-idf']

  q_norm_factor = math.sqrt(q_norm_factor)

  for term in query_dict:
    query_dict[term]['tf-idf'] = query_dict[term]['tf-idf']/q_norm_factor

  #topic modelling of query
  bow_vector = dictionary.doc2bow(query)
  query_lda = {}

  for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    query_lda.update({index:score})
  
  #calculating cosine similarity


  result = {} #to store all the documents as result and their cosine similarity score

  for query_term in query_dict:

    for doc_num in master_dict[query_term]:

      if doc_num in result:
        result[doc_num] += query_dict[query_term]['tf-idf']*master_dict[query_term][doc_num]['tf-idf']
      else:
        result.update({doc_num: 0.1})
        result[doc_num] = query_dict[query_term]['tf-idf']*master_dict[query_term][doc_num]['tf-idf']

  #calculating LDA score for each doc in results
  result_LDA = {}

  for doc_num in result:
    result_LDA.update({doc_num:0})

    for topic_id in query_lda:

      if topic_id in doc_lda[doc_num]:
        #dot product of lda scores
        result_LDA[doc_num] += query_lda[topic_id]*doc_lda[doc_num][topic_id]

  #extracting results with LDA
  for doc_num in result:
    result[doc_num] += result_LDA[doc_num]/3

  sorted_result_lda = sorted(result.items(), key=operator.itemgetter(1),reverse=True)
  
  for ind in range(10):
    print(comb_news[sorted_result_lda[ind][0]])

  result_s = []
  for ind in range(10):
    cont = []
    cont.append(comb_news[sorted_result_lda[ind][0]])
    cont.append(links[sorted_result_lda[ind][0]])
    result_s.append(cont)
  
  return result_s

# Flask Application

In [17]:
!pip install flask-ngrok

In [19]:
from flask_ngrok import run_with_ngrok
from flask import render_template,request
from flask import Flask

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/")
def give_query():
    return render_template("index.html")

@app.route('/results',methods=["POST"])
def get_result():
	query = request.form.get('query')
	if not query:
		return "Query not entered"
	result_s = getResults(query)
	# for ind in range(10):
	# 	result_s.append(comb_news[sorted_result_lda[ind][0]])

	return render_template("result.html",q=query,res=result_s)
  
def home():
    return "<h1>Running Flask on Google Colab!</h1>"

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ce4f54d7ca7c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Oct/2020 10:05:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2020 10:05:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2020 10:05:45] "GET /static/master.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2020 10:05:45] "GET /favicon.ico HTTP/1.1" 404 -


['trump', 'kim']


127.0.0.1 - - [25/Oct/2020 10:08:30] "POST /results HTTP/1.1" 200 -


Leave It To Kim Kardashian To Dress Up Like Kim Kardashian For Halloween. She would.
We Have A Lot Of Questions About Kim Kardashian's Latest Outfit. EXPLAIN YOURSELF, KIM.
All Of Kim Kardashian's Major Magazine Covers (PHOTOS). Stefano Tonchi, editor-in-chief of W magazine put Kim on the cover for his November 2010 issue. When asked why he chose Kim
Kim Kardashian Cleavage Also Shows Off Bra (PHOTOS). See Kim's style through the years... Oh well. At least she's wearing one. In other Kim Kardashian News, we are pleased to
Kim Kardashian Knows How To Dress Her Curves (PHOTO). Click here to see Kim Kardashian's style evolution. WHO: TV personality, entrepreneur and Kanye's better half, Kim Kardashian
'Kim Possible' Live-Action Movie Finds Its New Kim And Ron. Call them, beep them if you wanna reach them.
Donald Trump Will Meet With Kim Jong Un By May, South Korea Says. Kim reportedly has expressed his "eagerness to meet" with Trump as soon as possible.
Kim Kardashian's Surprising Outfit:

127.0.0.1 - - [25/Oct/2020 10:08:30] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2020 10:09:45] "POST /results HTTP/1.1" 200 -


['trump', 'kim']
Leave It To Kim Kardashian To Dress Up Like Kim Kardashian For Halloween. She would.
We Have A Lot Of Questions About Kim Kardashian's Latest Outfit. EXPLAIN YOURSELF, KIM.
All Of Kim Kardashian's Major Magazine Covers (PHOTOS). Stefano Tonchi, editor-in-chief of W magazine put Kim on the cover for his November 2010 issue. When asked why he chose Kim
Kim Kardashian Cleavage Also Shows Off Bra (PHOTOS). See Kim's style through the years... Oh well. At least she's wearing one. In other Kim Kardashian News, we are pleased to
Kim Kardashian Knows How To Dress Her Curves (PHOTO). Click here to see Kim Kardashian's style evolution. WHO: TV personality, entrepreneur and Kanye's better half, Kim Kardashian
'Kim Possible' Live-Action Movie Finds Its New Kim And Ron. Call them, beep them if you wanna reach them.
Donald Trump Will Meet With Kim Jong Un By May, South Korea Says. Kim reportedly has expressed his "eagerness to meet" with Trump as soon as possible.
Kim Kardashian's S

127.0.0.1 - - [25/Oct/2020 10:10:18] "GET /robots.txt HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2020 10:10:18] "GET /robots.txt HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2020 10:10:18] "GET /ads.txt HTTP/1.1" 404 -
